In [5]:
import pandas as pd #导入Pandas
import numpy as np #导入Numpy
import jieba #导入结巴分词
jieba.enable_parallel(32)

from keras.preprocessing import sequence
from keras.optimizers import SGD, RMSprop, Adagrad
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM, GRU

from __future__ import absolute_import #导入3.x的特征函数
from __future__ import print_function

In [3]:
neg=pd.read_excel('neg.xls',header=None,index=None)
pos=pd.read_excel('pos.xls',header=None,index=None) #读取训练语料完毕
pos['mark']=1
neg['mark']=0 #给训练语料贴上标签
pn=pd.concat([pos,neg],ignore_index=True) #合并语料
neglen=len(neg)
poslen=len(pos) #计算语料数目

In [4]:
cw = lambda x: list(jieba.cut(x)) #定义分词函数
pn['words'] = pn[0].apply(cw)

comment = pd.read_excel('sum.xls') #读入评论内容
#comment = pd.read_csv('a.csv', encoding='utf-8')
comment = comment[comment['rateContent'].notnull()] #仅读取非空评论
comment['words'] = comment['rateContent'].apply(cw) #评论分词 

d2v_train = pd.concat([pn['words'], comment['words']], ignore_index = True) 

Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
Loading model cost 1.918 seconds.
Prefix dict has been built succesfully.


In [6]:
w = [] #将所有词语整合在一起
for i in d2v_train:
  w.extend(i)

dict = pd.DataFrame(pd.Series(w).value_counts()) #统计词的出现次数
del w,d2v_train
dict['id']=list(range(1,len(dict)+1))

get_sent = lambda x: list(dict['id'][x])
pn['sent'] = pn['words'].apply(get_sent) #速度太慢

maxlen = 50

print("Pad sequences (samples x time)")
pn['sent'] = list(sequence.pad_sequences(pn['sent'], maxlen=maxlen))

Pad sequences (samples x time)


In [7]:
x = np.array(list(pn['sent']))[::2] #训练集
y = np.array(list(pn['mark']))[::2]
xt = np.array(list(pn['sent']))[1::2] #测试集
yt = np.array(list(pn['mark']))[1::2]
xa = np.array(list(pn['sent'])) #全集
ya = np.array(list(pn['mark']))

In [8]:
print('Build model...')
model = Sequential()
model.add(Embedding(len(dict)+1, 256))
model.add(LSTM(128)) # try using a GRU instead, for fun

Build model...


In [9]:

model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', class_mode="binary")

In [10]:
from keras.utils import plot_model
plot_model(model, to_file='model.png')

ImportError: Failed to import pydot. You must install pydot and graphviz for `pydotprint` to work.

In [ ]:
model.fit(xa, ya, batch_size=16, nb_epoch=10) #训练时间为若干个小时

In [ ]:
classes = model.predict_classes(xa)
acc = np_utils.accuracy(classes, ya)
print('Test accuracy:', acc)

In [3]:
# -*- coding:utf-8 -*-
'''
使用训练语料和扩展语料构造一个词典后，
构建LSTM网络训练情感分类模型
'''
from __future__ import absolute_import #导入3.x的特征函数
from __future__ import print_function
import pandas as pd #导入Pandas
import numpy as np #导入Numpy
import jieba #导入结巴分词
import h5py
import sys
import gensim
import keras

from keras.preprocessing import sequence
from keras.optimizers import SGD, RMSprop, Adagrad
from keras.utils import plot_model
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM, GRU
from keras.layers.wrappers import Bidirectional
from keras.callbacks import TensorBoard
from keras.metrics import binary_accuracy
from keras import backend as K
from pickle import dump, load
# import json

Using TensorFlow backend.


In [15]:
margin = 0.6
theta = lambda t: (K.sign(t)+1.)/2.

print('Build from raw data.')
jieba.enable_parallel(32)
data = pd.read_excel("sentiment_data.xlsx")
pn = pd.DataFrame()
pn['sent'] = data.sentence
pn['mark'] = data.emotion_1
cw = lambda x: list(jieba.cut(str(x))) #定义分词函数
pn['words'] = pn['sent'].apply(cw)
d2v_train = pn['words']
w = [] #将所有词语整合在一起
for i in d2v_train:
    w.extend(i)
dict_w = pd.DataFrame(pd.Series(w).value_counts()) #统计词的出现次数
del w,d2v_train
dict_w['id']=list(range(1,len(dict_w)+1))

get_sent = lambda x: list(dict_w['id'][x])
pn['sent'] = pn['words'].apply(get_sent) #速度太慢
maxlen = 50
print("Pad sequences (samples x time)")
pn['sent'] = list(sequence.pad_sequences(pn['sent'], maxlen=maxlen, padding='post', truncating='post'))

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache


Build from raw data.


Loading model cost 1.251 seconds.
Prefix dict has been built succesfully.


Pad sequences (samples x time)


In [7]:
margin = 0.6
theta = lambda t: (K.sign(t)+1.)/2.

data = pd.read_excel("sentiment_data.xlsx")

13

In [31]:
a = emotion_encoder(emotion)
a = a[2:]
len(a)
print(a)
emo = []
emo_reverse_dict = dict(zip(range(8), ['none', 'disgust', 'like', 'happiness', 'sadness', 'surprise', 'anger', 'fear']))
for i in range(len(a)):
    if a[i]=='1':
        emo.append(emo_reverse_dict[i])
emo

10001101


['none', 'sadness', 'surprise', 'fear']

In [42]:
emotion = ['happiness','like','none','fear']
def emotion_list_encoder(emo_list):
    emo_dict = dict(zip(['none', 'disgust', 'like', 'happiness', 'sadness', 'surprise', 'anger', 'fear'], range(8)))
    emo_encrypt_list = [2**emo_dict[x] for x in emo_list]
    code = bin(sum(emo_encrypt_list))
    return code
print(emotion_encoder(emotion))
def emotion_list_decoder(emo_index):
    emotion_index = emo_index[2:]
    emo_reverse_dict = dict(zip(range(8), ['none', 'disgust', 'like', 'happiness', 'sadness', 'surprise', 'anger', 'fear']))
    emo_list = []
    for i in range(len(emotion_index)):
        if emotion_index[i]=='1':
            emo_list.append(emo_reverse_dict[i])
    return emo_list
print(emotion_list_decoder(emotion_list_encoder(emotion)))

0b10001101
['none', 'sadness', 'surprise', 'fear']


In [37]:
print([(a,b) for a in range(5) for b in range(5)])
data.shape[0]

[(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (1, 0), (1, 1), (1, 2), (1, 3), (1, 4), (2, 0), (2, 1), (2, 2), (2, 3), (2, 4), (3, 0), (3, 1), (3, 2), (3, 3), (3, 4), (4, 0), (4, 1), (4, 2), (4, 3), (4, 4)]


16706

In [44]:
data['mark'] = [emotion_list_encoder([data.emotion_1[i], data.emotion2[i]]) for i in range(data.shape[0])]
data['raw_emotion'] = data['mark'].apply(emotion_list_decoder)
data

,sentence,emotion_1,emotion2,mark,raw_emotion
0,今儿老爸逮着我一顿狠念！,none,none,0b10,[none]
1,念得我各种烦躁……,disgust,none,0b11,"[none, disgust]"
2,我要不要考虑下降低回家的频率啊 回来一次吵一次= =真烦人！！！,disgust,none,0b11,"[none, disgust]"
3,在考虑是否要回刷2.3Rom，黑/锁屏状态，屏幕微微亮;,none,none,0b10,[none]
4,发烫耗电快…,none,none,0b10,[none]
5,这让我蛋疼！,disgust,none,0b11,"[none, disgust]"
6,不过，喜欢4.0原生态的界面！,like,none,0b101,"[none, like]"
7,去年的今天我去上了政治辅导班，回来看芒果台的跨年晚会，然后叠了纸飞机在上面许下了心愿。,none,none,0b10,[none]
8,又要重复我每年必说的台词了，2010就要过去了，我很怀念它。,like,none,0b101,"[none, like]"
9,至少，它看上去很美。,like,none,0b101,"[none, like]"


In [51]:
len(data.mark.unique())
labels = data.mark.unique()
type(labels)
trans_dict = dict(zip(labels, range(len(labels))))
rev_trans_dict = dict(zip(range(len(labels)), labels))
trans_dict
trans_fun = lambda x:trans_dict[x]
rev_trans_dict
data['new_mark'] = data['mark'].apply(trans_fun)

,sentence,emotion_1,emotion2,mark,raw_emotion,new_mark
0,今儿老爸逮着我一顿狠念！,none,none,0b10,[none],0
1,念得我各种烦躁……,disgust,none,0b11,"[none, disgust]",1
2,我要不要考虑下降低回家的频率啊 回来一次吵一次= =真烦人！！！,disgust,none,0b11,"[none, disgust]",1
3,在考虑是否要回刷2.3Rom，黑/锁屏状态，屏幕微微亮;,none,none,0b10,[none],0
4,发烫耗电快…,none,none,0b10,[none],0
5,这让我蛋疼！,disgust,none,0b11,"[none, disgust]",1
6,不过，喜欢4.0原生态的界面！,like,none,0b101,"[none, like]",2
7,去年的今天我去上了政治辅导班，回来看芒果台的跨年晚会，然后叠了纸飞机在上面许下了心愿。,none,none,0b10,[none],0
8,又要重复我每年必说的台词了，2010就要过去了，我很怀念它。,like,none,0b101,"[none, like]",2
9,至少，它看上去很美。,like,none,0b101,"[none, like]",2


In [85]:
trans_dict1 = dict(zip(data.mark.unique(),range(len(data.mark.unique()))))
trans_emo1 = lambda x:trans_dict1[x]
data['mark'] = data['mark'].apply(trans_emo1)
data

,sentence,emotion_1,emotion2,mark
0,今儿老爸逮着我一顿狠念！,1,1,0
1,念得我各种烦躁……,2,1,1
2,我要不要考虑下降低回家的频率啊 回来一次吵一次= =真烦人！！！,2,1,1
3,在考虑是否要回刷2.3Rom，黑/锁屏状态，屏幕微微亮;,1,1,0
4,发烫耗电快…,1,1,0
5,这让我蛋疼！,2,1,1
6,不过，喜欢4.0原生态的界面！,4,1,2
7,去年的今天我去上了政治辅导班，回来看芒果台的跨年晚会，然后叠了纸飞机在上面许下了心愿。,1,1,0
8,又要重复我每年必说的台词了，2010就要过去了，我很怀念它。,4,1,2
9,至少，它看上去很美。,4,1,2


In [73]:
len(data.mark.unique())

29

In [8]:
# margin = 0.6
# theta = lambda t: (K.sign(t)+1.)/2.

jieba.enable_parallel(32)
data = pd.read_excel("sentiment_data.xlsx")
trans_dict = dict(zip(data.emotion_1.unique(), range(8)))
trans_emo = lambda x:trans_dict[x]
data['mark'] = data['emotion_1'].apply(trans_emo)
pn = pd.DataFrame()
pn['sent'] = data.sentence
pn['mark'] = data.mark
cw = lambda x: list(jieba.cut(str(x))) #定义分词函数
pn['words'] = pn['sent'].apply(cw)

print('try loading pretrained word2vec model.')
wordvec_model = gensim.models.word2vec.Word2Vec.load('/home/da/nlp_repo/rnn_test/word2vec_wx/word2vec_wx')
wordvec_weight = wordvec_model.wv.syn0
vocab = dict([(k, v.index) for k, v in wordvec_model.wv.vocab.items()])
word_embedding_layer = Embedding(
        input_dim=wordvec_weight.shape[0],
        output_dim=wordvec_weight.shape[1],
        weights=[wordvec_weight],
        trainable=True)
word_to_id = lambda word: not (vocab.get(word) is None) and vocab.get(word) or 0
words_to_ids = lambda words: list(map(word_to_id, words))
pn['sent'] = pn['words'].apply(words_to_ids) #速度太慢
maxlen = 50
reverse_seq = lambda id_seq: id_seq[::-1]
concat_seq = lambda a,b: list(np.hstack((a, b)))
print("Pad sequences (samples x time)")
pn['sent_rev'] = list(sequence.pad_sequences(pn['sent'], maxlen=maxlen))
pn['sent_rev'] = pn['sent_rev'].apply(reverse_seq)
pn['sent'] = list(sequence.pad_sequences(pn['sent'], maxlen=maxlen, padding='post', truncating='post'))
pn['sent'] = pn['sent'].combine(pn['sent_rev'], func=concat_seq)

try loading pretrained word2vec model.
Pad sequences (samples x time)


In [9]:
num_classes = len(data.mark.unique())
x = np.array(list(pn['sent']))[::2] #训练集
y = np.array(list(pn['mark']))[::2]

In [10]:
y = keras.utils.to_categorical(y, num_classes)
xt = np.array(list(pn['sent']))[1::2] #测试集
yt = np.array(list(pn['mark']))[1::2]
yt = keras.utils.to_categorical(yt, num_classes)
xa = np.array(list(pn['sent'])) #全集
ya = np.array(list(pn['mark']))

In [19]:
print('Build model...')
model = Sequential()
model.add(Embedding(len(dict_w)+1, 256))
#model.add(Bidirectional(LSTM(128)))
model.add(GRU(128))  # try using a GRU instead, for fun
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('softmax'))
improved_loss = lambda y_true, y_pred: -(1-theta(y_true-margin)*theta(y_pred-margin)-theta(1-margin-y_true)*theta(1-margin-y_pred))*(y_true*K.log(y_pred)+(1-y_true)*K.log(1-y_pred))
# using the user defined improved loss fun
model.compile(loss=improved_loss, optimizer='adam', metrics=['mae', 'acc'])
# using the default binary crossentropy loss fun
#model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['mae', 'acc'])

Build model...


In [11]:
print('Build model...')
model = Sequential()
model.add(word_embedding_layer)
model.add(Dropout(0.1))
# model.add(Bidirectional(LSTM(128))) # for looped input sentence sequence, no need Bidirectional
#model.add(LSTM(128))
model.add(GRU(128))
# model.add(LSTM(256, 128)) # try using a GRU instead, for fun
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('softmax'))
#improved_loss = lambda y_true, y_pred: -(1-theta(y_true-margin)*theta(y_pred-margin)-theta(1-margin-y_true)*theta(1-margin-y_pred))*(y_true*K.log(y_pred)+(1-y_true)*K.log(1-y_pred))
# using the user defined improved loss fun
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['mae', 'acc'])

Build model...


In [12]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, None, 256)         90162176  
_________________________________________________________________
dropout_1 (Dropout)          (None, None, 256)         0         
_________________________________________________________________
gru_1 (GRU)                  (None, 128)               147840    
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 129       
_________________________________________________________________
activation_1 (Activation)    (None, 1)                 0         
Total params: 90,310,145.0
Trainable params: 90,310,145.0
Non-trainable params: 0.0
__________________________________________________________

In [32]:
model.fit(x, y, batch_size=16, epochs=10, validation_data=(xt, yt)) #训练时间为若干个小时 # avg 440s per epoch on server with 32 cores.

Train on 8353 samples, validate on 8353 samples
Epoch 1/10


ValueError: could not convert string to float: 'none'

In [ ]:
scores = model.evaluate(xt, yt, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
print("%s: %.2f%%" % (model.metrics_names[2], scores[2]*100))

In [ ]:
model.save('models/Bi_lstm_senti_0406_improve_loss_post.h5')
json_string = model.to_json()
with open('models/Bi_lstm_senti_0406_improve_loss_post.json', 'wt') as f:
    f.write(json_string)
model.save_weights('models/Bi_lstm_senti_0406_improve_loss_post_weights.h5')
print('Model struct and weights saved.')
#classes = model.predict(xt)
#acc = np_utils.accuracy(classes, yt)
#variable(value, dtype='float32', name=None))
#acc = binary_accuracy(variable(yt, dtype='float64'), variable(classes, dtype='float64'))
#print('Test accuracy:', acc)
pred_x = np.array(list(comment['sent']))
pred_y = model.predict(pred_x)
comment['mark']=list(pred_y)
dump(comment, open('cache/comment_senti_predict.pkl', 'wb'))
writer = pd.ExcelWriter('output/predict_comment_senti.xlsx')
comment.to_excel(writer, sheet_name='senti', encoding = 'utf-8', index = False)
writer.save()

In [54]:
import pymysql

def get_db_data(query_str):
        conn = pymysql.connect(host='127.0.0.1',port=3306,user='analyzer',password='analyzer@tbs2016',database='dp_relation',charset='utf8mb4',cursorclass=pymysql.cursors.DictCursor)
        cur=conn.cursor()
        doc = pd.read_sql_query(query_str, conn)
        doc = pd.DataFrame(doc)
        for column in doc.columns:
            column_null = pd.isnull(doc[column])
            column_null_true = column_null[column_null == True]
            if len(column_null_true) == len(doc):
                del doc[column]
        cur.close()
        conn.close()
        return doc

test_data = get_db_data("select * from weibo_raw_data where keyword_id='37_5'")
test_data.loc[0:10,'content']
comment = test_data

In [56]:
jieba.enable_parallel(32)
cw = lambda x: list(jieba.cut(str(x))) #定义分词函数
maxlen = 50
print('try loading pretrained word2vec model.')
wordvec_model = gensim.models.word2vec.Word2Vec.load('/home/da/nlp_repo/rnn_test/word2vec_wx/word2vec_wx')
wordvec_weight = wordvec_model.wv.syn0
vocab = dict([(k, v.index) for k, v in wordvec_model.wv.vocab.items()])
word_to_id = lambda word: not (vocab.get(word) is None) and vocab.get(word) or 0
words_to_ids = lambda words: list(map(word_to_id, words))
reverse_seq = lambda id_seq: id_seq[::-1]
# concat_seq = lambda df: np.concatenate((df.sent, df.sent_rev))
concat_seq = lambda a,b: list(np.hstack((a, b)))
comment = comment[comment['content'].notnull()] #仅读取非空评论
comment['words'] = comment['content'].apply(cw) #评论分词
comment['sent'] = comment['words'].apply(words_to_ids)
comment['sent_rev'] = list(sequence.pad_sequences(comment['sent'], maxlen=maxlen, padding='pre', truncating='pre'))
comment['sent_rev'] = comment['sent_rev'].apply(reverse_seq)
comment['sent'] = list(sequence.pad_sequences(comment['sent'], maxlen=maxlen, padding='post', truncating='post'))
# print(comment['sent'].shape, comment['sent_rev'].shape)
comment['sent'] = comment['sent'].combine(comment['sent_rev'], func=concat_seq)
print(comment['sent'][0])
# comment['sent'] = comment.apply(concat_seq, axis=1, raw=True, reduce=True)
comment

try loading pretrained word2vec model.
[411, 121, 1, 35, 25284, 5, 30244, 3465, 18, 5, 125790, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 125790, 5, 18, 3465, 30244, 5, 25284, 35, 1, 121, 411, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


,id,project_id,project_name,keyword_id,keyword,user_id,content,created_at,url,comment_count,attitude_count,repost_count,source,words,sent,sent_rev
0,2077339,37,美素化妆品_170411,37_5,倩碧_消费者用户微博原创,130788,这么小的一个电阻 510欧！ ​,2017-04-09 17:45:00 +0800,http://weibo.com/2076419035/EDJ2F3kDF,0,2,0,Weibo.intl,"[这么, 小, 的, 一个, 电阻, , 510, 欧, ！, , ​]","[411, 121, 1, 35, 25284, 5, 30244, 3465, 18, 5...","[125790, 5, 18, 3465, 30244, 5, 25284, 35, 1, ..."
1,2077340,37,美素化妆品_170411,37_5,倩碧_消费者用户微博原创,130788,我以后再也不相信女的不会秃顶这种屁话了。 ​,2017-04-09 13:50:48 +0800,http://weibo.com/2076419035/EDHvBb6hB,2,0,0,Weibo.intl,"[我, 以后, 再也, 不, 相信, 女, 的, 不会, 秃顶, 这种, 屁话, 了, 。,...","[20, 563, 2919, 19, 702, 807, 1, 236, 42043, 2...","[125790, 5, 3, 7, 98264, 241, 42043, 236, 1, 8..."
2,2077341,37,美素化妆品_170411,37_5,倩碧_消费者用户微博原创,130788,护肤然后害熬夜 是不是有病啊 ​,2017-04-09 04:01:35 +0800,http://weibo.com/2076419035/EDDEr4Iz5,1,1,0,Weibo.intl,"[护肤, 然后, 害, 熬夜, , 是不是, 有病, 啊, , ​]","[2426, 372, 5979, 4361, 5, 854, 0, 332, 5, 125...","[125790, 5, 332, 0, 854, 5, 4361, 5979, 372, 2..."
3,2077342,37,美素化妆品_170411,37_5,倩碧_消费者用户微博原创,130788,现在竟然还有这个 我感觉这个是卖烤冷面的大爷用来呲水的 ​,2017-04-08 12:39:10 +0800,http://weibo.com/2076419035/EDxC1wPvC,3,1,0,Weibo.intl,"[现在, 竟然, 还有, 这个, , 我, 感觉, 这个, 是, 卖, 烤, 冷面, 的,...","[199, 2191, 230, 101, 5, 20, 393, 101, 6, 1012...","[125790, 5, 1, 281, 44042, 2227, 8540, 1, 3041..."
4,2077343,37,美素化妆品_170411,37_5,倩碧_消费者用户微博原创,130788,他走得好慢，好驼，好弱。好苦。\n我看着他过马路，既为他担心，也觉得那样缓慢孤独被遗弃的命运...,2017-04-06 23:49:40 +0800,http://weibo.com/2076419035/EDj9cc4vm,0,3,0,Weibo.intl,"[他, 走, 得, 好慢, ，, 好, 驼, ，, 好弱, 。, 好苦, 。, \n, 我,...","[39, 261, 127, 0, 0, 58, 28550, 0, 0, 3, 0, 3,...","[125790, 5, 3, 379, 1, 2861, 42, 75, 0, 2833, ..."
5,2077344,37,美素化妆品_170411,37_5,倩碧_消费者用户微博原创,130788,我估计是我天天想着为啥我这么倒霉然后我就倒霉了。以后我要天天高歌我真幸运！我真幸运！ ​,2017-04-04 17:31:54 +0800,http://weibo.com/2076419035/ECXORjMev,2,0,0,Weibo.intl,"[我, 估计, 是, 我, 天天, 想着, 为啥, 我, 这么, 倒霉, 然后, 我, 就,...","[20, 3642, 6, 20, 2026, 3980, 7967, 20, 411, 1...","[125790, 5, 18, 3337, 704, 20, 18, 3337, 704, ..."
6,2077345,37,美素化妆品_170411,37_5,倩碧_消费者用户微博原创,130788,为啥出租车要按公里算钱呢？按时间算是不能走的快点儿？？ ​,2017-04-04 17:18:57 +0800,http://weibo.com/2076419035/ECXJBnX1N,5,0,0,Weibo.intl,"[为啥, 出租车, 要, 按, 公里, 算钱, 呢, ？, 按, 时间, 算是, 不能, 走...","[7967, 5287, 31, 112, 1259, 0, 179, 36, 112, 1...","[125790, 5, 36, 36, 103617, 1, 261, 182, 3704,..."
7,2077346,37,美素化妆品_170411,37_5,倩碧_消费者用户微博原创,130788,东北一家人太好看了！！！大家快追起来！！ ​,2017-04-04 15:12:20 +0800,http://weibo.com/2076419035/ECWUd9Hxl,4,0,0,Weibo.intl,"[东北, 一家人, 太, 好看, 了, ！, ！, ！, 大家, 快追, 起来, ！, ！,...","[3184, 6477, 217, 2412, 7, 18, 18, 18, 151, 27...","[125790, 5, 18, 18, 311, 275007, 151, 18, 18, ..."
8,2077347,37,美素化妆品_170411,37_5,倩碧_消费者用户微博原创,130788,我喜欢的表情的排名\n1号可爱表情：[熊猫]2:[兔子]3:[羞嗒嗒]4:[傻眼] ​,2017-04-04 00:40:31 +0800,http://weibo.com/2076419035/ECRcl7C59,4,0,0,Weibo.intl,"[我, 喜欢, 的, 表情, 的, 排名, \n, 1, 号, 可爱, 表情, ：, [, ...","[20, 233, 1, 3478, 1, 2431, 2, 44, 190, 2001, ...","[125790, 5, 1152, 35844, 1149, 47, 99, 1152, 6..."
9,2077348,37,美素化妆品_170411,37_5,倩碧_消费者用户微博原创,130788,我好朋友真的讲话都挺厉害的还。我记得我高中去看他辩论会的时候，同学A举起两张白纸说：假设这是...,2017-04-04 00:30:28 +0800,http://weibo.com/2076419035/ECR8ganjm,2,0,0,Weibo.intl,"[我, 好, 朋友, 真的, 讲话, 都, 挺, 厉害, 的, 还, 。, 我, 记得, 我...","[20, 58, 215, 474, 3287, 24, 1992, 3191, 1, 65...","[125790, 5, 3, 158, 78, 20, 6, 25185, 0, 11626..."


In [59]:
rev_trans_dict = load(open('/home/jeffmxh/rnn_test/models/bi_trans_dict.pickle','rb'))

In [62]:
from keras.models import Sequential, load_model, model_from_json

print('data ready. begin predicting.')
# model = load_model('Bi_lstm_senti_0406_improve_loss_post.h5')
with open('/home/jeffmxh/rnn_test/models/combine_emo_GRU_senti_0420.json', 'rt') as f:
    json_string=f.read()
model = model_from_json(json_string)
model.load_weights('/home/jeffmxh/rnn_test/models/combine_emo_LSTM_senti_0420_weights.h5')

pred_x = np.array(list(comment['sent']))
pred_y = model.predict(pred_x)
comment['mark']=list(pred_y)
comment.loc[0:5,['content','mark']]

data ready. begin predicting.


,content,mark
0,这么小的一个电阻 510欧！ ​,"[0.531539, 0.0223912, 0.191103, 0.0218594, 0.0..."
1,我以后再也不相信女的不会秃顶这种屁话了。 ​,"[0.0980624, 0.215746, 0.0409109, 0.0103792, 0...."
2,护肤然后害熬夜 是不是有病啊 ​,"[0.191436, 0.184627, 0.0376403, 0.00470487, 0...."
3,现在竟然还有这个 我感觉这个是卖烤冷面的大爷用来呲水的 ​,"[0.0629487, 0.0226193, 0.102648, 0.088204, 0.0..."
4,他走得好慢，好驼，好弱。好苦。\n我看着他过马路，既为他担心，也觉得那样缓慢孤独被遗弃的命运...,"[0.826319, 0.00783627, 0.0649442, 0.00315705, ..."
5,我估计是我天天想着为啥我这么倒霉然后我就倒霉了。以后我要天天高歌我真幸运！我真幸运！ ​,"[0.0322943, 0.113635, 0.0139241, 0.00496496, 0..."


In [79]:
def emotion_list_decoder(emo_index):
    emotion_index = emo_index[2:]
    emo_reverse_dict = dict(zip(range(8), ['none', 'disgust', 'like', 'happiness', 'sadness', 'surprise', 'anger', 'fear']))
    emo_list = []
    for i in range(len(emotion_index)):
        if emotion_index[i]=='1':
            emo_list.append(emo_reverse_dict[i])
    return emo_list
def result_translate(prob_list):
    sign_dict = dict(zip(prob_list, rev_trans_dict.values()))
    result_emotion_code = sign_dict[max(prob_list)]
    return emotion_list_decoder(result_emotion_code)

comment['emotion_result'] = comment['mark'].apply(result_translate)
comment = comment.loc[:,['content','emotion_result']]
comment
#result_translate(comment.mark[3])

,content,emotion_result
0,这么小的一个电阻 510欧！ ​,[none]
1,我以后再也不相信女的不会秃顶这种屁话了。 ​,"[none, anger]"
2,护肤然后害熬夜 是不是有病啊 ​,"[none, anger]"
3,现在竟然还有这个 我感觉这个是卖烤冷面的大爷用来呲水的 ​,"[none, happiness]"
4,他走得好慢，好驼，好弱。好苦。\n我看着他过马路，既为他担心，也觉得那样缓慢孤独被遗弃的命运...,[none]
5,我估计是我天天想着为啥我这么倒霉然后我就倒霉了。以后我要天天高歌我真幸运！我真幸运！ ​,"[none, anger]"
6,为啥出租车要按公里算钱呢？按时间算是不能走的快点儿？？ ​,"[none, disgust]"
7,东北一家人太好看了！！！大家快追起来！！ ​,"[none, like]"
8,我喜欢的表情的排名\n1号可爱表情：[熊猫]2:[兔子]3:[羞嗒嗒]4:[傻眼] ​,"[none, happiness]"
9,我好朋友真的讲话都挺厉害的还。我记得我高中去看他辩论会的时候，同学A举起两张白纸说：假设这是...,[none]


In [81]:
# dump(comment, open('comment_senti_predict.pkl', 'wb'))
print('save result to excel...')
writer = pd.ExcelWriter('37_5predict_emotion.xlsx')
comment.to_excel(writer, sheet_name='senti', encoding = 'utf-8', index = False)
writer.save()
print('task done.')

save result to excel...
task done.
